# !pip install

In [ ]:
! pip install unidecode

     |████████████████████████████████| 245kB 5.0MB/s 


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=618d9576bd88c8f46942b1e08e542b97f27311afb93abf3e2a588454f403583f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_rtk5db7/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 5.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617549 sha256=04262501d599f879a1c2d0031113004b63bfaf3565914eb66c1032ea73fc9b01
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
# !pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

import sqlite3

from google.colab import files

In [ ]:
import surprise
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise import similarities
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

import spacy
from spacy.tokenizer import Tokenizer


mport matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:

import random

my_seed = 42
random.seed(my_seed)
np.random.seed(my_seed)

# user playlist 데이터셋

[Exploiting Music Play Sequence for Music Recommendation - CORE Reader](https://core.ac.uk/reader/194453000) 논문에 따르면 유저가 한 세션 내에서 플레이한 곡들은 유사도가 크다고 한다. 그렇기 때문에 유저별 플레이 데이터를 하나의 플레이리스트로 보기로 한다.

In [ ]:
text = '/content/drive/MyDrive/capstone project 1/train_triplets.txt'

In [ ]:
# 곡 재생횟수 데이터셋
df = pd.read_csv(text,sep='\t', names = ['user_id', 'song_id', 'play_counts'])

In [ ]:
df.head()

,user_id,song_id,play_counts
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [ ]:
print(df.shape)
print(df.isnull().sum())

(48373586, 3)
user_id        0
song_id        0
play_counts    0
dtype: int64


In [ ]:
# user_id 별로 정렬하기 위해 sort
df_sorted = df.sort_values(by='user_id', ascending=False)

In [ ]:
df_sorted.head()

,user_id,song_id,play_counts
31292919,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1
31292934,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1
31292932,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,32
31292931,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1
31292930,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,56


In [ ]:
# 유저별 플레이한 곡 수 세기 위해 groupby 사용
df_groupby = df_sorted.groupby('user_id').nunique()

In [ ]:
df_groupby = df_groupby.drop(columns=['play_counts'])

In [ ]:
df_groupby

,song_id
user_id,
00000b722001882066dff9d2da8a775658053ea0,12
00001638d6189236866af9bbf309ae6c2347ffdc,14
0000175652312d12576d9e6b84f600caa24c4715,12
00001cf0dce3fb22b0df0f3a1d9cd21e38385372,18
0000267bde1b3a70ea75cf2b2d216cb828e3202b,25
...,...
ffffdc6c89988cd6119067769162948eacf8b670,51
ffffe07df4bb5fd929efe42c5728f3a0c1621277,14
fffff3e690fcda840b716ce7249d8935ff3323fc,15


In [ ]:
df_groupby['song_id'].describe() # 여기서 song_id 컬럼은 유저별 플레이리스트 내 곡 수 

count    1.019318e+06
mean     4.745682e+01
std      5.781573e+01
min      1.000000e+01
25%      1.600000e+01
50%      2.700000e+01
75%      5.500000e+01
max      4.400000e+03
Name: song_id, dtype: float64

평균적으로 사용자별로 플레이한 곡 목록에 47개의 곡이 있음을 확인할 수 있다.

In [ ]:
df_sorted.shape

(48373586, 3)

In [ ]:
# df_sliced = df_sorted[:20000] # 데이터셋이 너무 크기 때문에 20000개로 줄이기

In [ ]:
# df_sliced_groupby = df_sliced.groupby('user_id').nunique() # 유저별 플레이한 곡 수 세기 위해 groupby 사용

In [ ]:
# df_sliced_groupby

,song_id,play_counts
user_id,,
ffe84b524f1f41e0da1d2f70e69c49d7c2b349e4,109,18
ffe85f9995f7221c636afde3c42e2566682bd829,199,16
ffe88447249c8cdd94410eba32d81cc97d6e9994,36,3
ffe89b15cc8c6bbc70876f5ce4822ee045a80317,35,5
ffe8d15e06162ddf0917441d88105258287dde01,12,8
...,...,...
ffffdc6c89988cd6119067769162948eacf8b670,51,8
ffffe07df4bb5fd929efe42c5728f3a0c1621277,14,1
fffff3e690fcda840b716ce7249d8935ff3323fc,15,6


In [ ]:
# df_sliced_groupby['song_id'].describe()

count    401.000000
mean      49.875312
std       56.496765
min       10.000000
25%       16.000000
50%       29.000000
75%       57.000000
max      472.000000
Name: song_id, dtype: float64

슬라이싱한 후 유저의 수 401명, 유저별 플레이한 곡 수 평균 49개, 최소 10개

In [ ]:
# 슬라이싱한 데이터프레임 csv로 다운로드
# df_sliced.to_csv('df_sliced.csv')

# files.download('df_sliced.csv')

In [ ]:
# 곡 정보를 불러오기 위해 슬라이싱한 후 데이터셋에서 재생된 곡들 저장
# unique_songs = df_sliced['song_id'].unique()

In [ ]:
# unique_songs.shape

(14143,)

In [ ]:
# filepath = '/content/drive/MyDrive/capstone project 1/df_sliced.csv'
# df_sliced = pd.read_csv(filepath)

401명의 유저가 재생한 곡 갯수는 14143개

# 곡 정보 메타데이터 셋

In [ ]:
meta_db = '/content/drive/MyDrive/capstone project 1/track_metadata.db'
con = sqlite3.connect(meta_db)
cursor = con.cursor()

In [ ]:
info = pd.read_sql_query("SELECT title, song_id, artist_name, release FROM songs", con)

info.head()

,title,song_id,artist_name,release
0,Silent Night,SOQMMHC12AB0180CB8,Faster Pussy cat,Monster Ballads X-Mas
1,Tanssi vaan,SOVFVAK12A8C1350D9,Karkkiautomaatti,Karkuteillä
2,No One Could Ever,SOGTUKN12AB017F4F1,Hudson Mohawke,Butter
3,Si Vos Querés,SOBNYVR12A8C13558C,Yerba Brava,De Culo
4,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Der Mystic,Rene Ablaze Presents Winter Sessions


In [ ]:
info.shape

(1000000, 4)

In [ ]:
info.to_csv('metadata.csv')

files.download('metadata.csv')

In [ ]:
# 재생된 곡(song_id) 별 title, artist값 합치기
# 빈 데이터프레임 생성
played_songId = pd.DataFrame(columns=['title', 'song_id', 'artist_name'])

for id in unique_songs:
  add_row = info.loc[info['song_id'] == id]
  played_songId = played_songId.append(add_row)

In [ ]:
played_songId.head()

,title,song_id,artist_name
74283,I Wanna Make You Close Your Eyes,SOFYCAM12A58A7A45B,Dierks Bentley
325856,Train Travelin',SOOOWXV12A6D4F4763,Dierks Bentley
187566,Breed,SONYEOJ12A8C142E86,Nirvana
456530,Rianna,SONHWUN12AC468C014,Fisher
611449,Le Jardin d'Hiver,SOMCMKG12A8C1347BF,Jacky Terrasson


In [ ]:
# 데이터셋 저장
played_songId.to_csv('played_songId.csv')

files.download('played_songId.csv')

# pitchfork 데이터셋

## 데이터 불러오기

In [ ]:
pitchfork_db = '/content/drive/MyDrive/capstone project 1/pitchfork.sqlite'
con_pitchfork = sqlite3.connect(pitchfork_db)

In [ ]:
cursor_pitchfork = con_pitchfork.cursor()

In [ ]:
pitchfork_df = pd.read_sql_query(
"SELECT title, artist, score,content.content FROM reviews \
JOIN content on content.reviewid = reviews.reviewid \
WHERE pub_year < 2012 GROUP BY reviews.reviewid", con_pitchfork)

곡 메타 데이터는 2011년까지의 자료만 있기 때문에 2012년 이전의 리뷰를 불러온다. 

In [ ]:
pitchfork_df.head()

,title,artist,score,content
0,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,..."
1,pure tone audiometry,aarktica,7.4,Can there be any purpose behind a master's deg...
2,or you could just go through your whole life a...,aarktica,8.1,Funny what gets tagged for popular comebacks t...
3,bleeding light,aarktica,7.2,Aarktica didn't seem to know what he was on 20...
4,abcs,abcs,8.0,ABCS are all about unity. Their bandname come...


In [ ]:
pitchfork_df.shape

(12425, 4)

In [ ]:
pitchfork_df['content'][1]

'Can there be any purpose behind a master\'s degree in the psychology of music other than inevitably attempting\n    to rule the minds of men through the manipulative powers of sound?  I submit that there can not; the job\n    prospects from a degree like that are, like, what?  Music therapist?  I don\'t think so.  The only career\n    I can imagine it successfully leading to is Supervillian.\n    \n    It doesn\'t take a superhuman intellect to see\n    through the aims of Jon DeRosa\'s education-- he\'s the arch-mastermind behind the warm, homey folk of Pale\n    Horse and Rider, and has guested on several other records, including Debridement, a recent Chairkickers\n    release by Rivulets.  But none of his various projects are more dangerous, more insidious, than the icy calm\n    of Aarktica\'s dreamy ambiance.  In fact, his nefarious desire to lull listeners into a pliant, peaceful\n    daydream (and then rob them, no doubt-- or worse!) could easily have succeeded with last year\'

In [ ]:
pitchfork_df.to_csv('pitchfork.csv')

files.download('pitchfork.csv')

## 데이터셋 합치기

In [ ]:
# 아티스트명과 타이틀 내 ()를 없애고 소문자로 변환한다..
info['artist_name'] = info['artist_name'].str.lower()
info['release'] = info['release'].str.lower()
info['artist_name'] = info['artist_name'].str.replace(r'\([^()]*\)', '')
info['release'] = info['release'].str.replace(r'\([^()]*\)', '')

In [ ]:
# artist_name -> artist컬럼명 변경
info = info.rename(columns={'release' : 'album', 'artist_name' : 'artist'})

In [ ]:
info

,title,song_id,artist,album
0,Silent Night,SOQMMHC12AB0180CB8,faster pussy cat,monster ballads x-mas
1,Tanssi vaan,SOVFVAK12A8C1350D9,karkkiautomaatti,karkuteillä
2,No One Could Ever,SOGTUKN12AB017F4F1,hudson mohawke,butter
3,Si Vos Querés,SOBNYVR12A8C13558C,yerba brava,de culo
4,Tangle Of Aspens,SOHSBXH12A8C13B0DF,der mystic,rene ablaze presents winter sessions
...,...,...,...,...
999995,O Samba Da Vida,SOTXAME12AB018F136,kiko navarro,pacha v.i.p.
999996,Jago Chhadeo,SOXQYIQ12A8C137FBB,kuldeep manak,naale baba lassi pee gya
999997,Novemba,SOHODZI12A8C137BB3,gabriel le mar,dub_connected: electronic music
999998,Faraday,SOLXGOR12A81C21EB7,elude,the trance collection vol. 2


In [ ]:
info.to_csv('info.csv')

files.download('info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pitchfork_df['artist'] = pitchfork_df['artist'].str.lower()
pitchfork_df['title'] = pitchfork_df['title'].str.lower()
pitchfork_df['artist'] = pitchfork_df['artist'].str.replace(r'\([^()]*\)', '')
pitchfork_df['title'] = pitchfork_df['title'].str.replace(r'\([^()]*\)', '')

In [ ]:
pitchfork_df = pitchfork_df.rename(columns={'title' : 'album'})

In [ ]:
pitchfork_df

,album,artist,score,content
0,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,..."
1,pure tone audiometry,aarktica,7.4,Can there be any purpose behind a master's deg...
2,or you could just go through your whole life a...,aarktica,8.1,Funny what gets tagged for popular comebacks t...
3,bleeding light,aarktica,7.2,Aarktica didn't seem to know what he was on 20...
4,abcs,abcs,8.0,ABCS are all about unity. Their bandname come...
...,...,...,...,...
12420,blue slide park,mac miller,1.0,"At the 2000 VMAs, Eminem's performance of ""The..."
12421,emika,emika,7.4,"The idea of a British-born, blue-eyed Berlinit..."
12422,hdcd box set,this mortal coil,9.0,"For a while in the 1980s, and to some degree i..."
12423,darkside ep,darkside,8.0,"Nicolas Jaar creates slow, strange, cloistered..."


In [ ]:
# 리뷰데이터와 곡메타데이터를 아티스트명과 제목으로 합친다.
merged_df = pd.merge(pitchfork_df, info, on=['artist', 'album'], how='inner')

In [ ]:
merged_df

,album,artist,score,content,title,song_id
0,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,...",Vegetarian Restaurant,SODDEDU12A8C140709
1,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,...",Surly Girl,SOZIXKT12A8C14071D
2,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,...",Heliopolis By Night,SOLWNEG12A58A7B67B
3,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,...",Something I Must Tell You,SOAHZSF12A58A7662E
4,young forever,aberfeldy,3.4,"Aberfeldy recorded their debut, Young Forever,...",Slow Me Down,SOPHUOP12A8C1406F7
...,...,...,...,...,...,...
31784,z,my morning jacket,7.6,"Here's an album that begs for vinyl, although ...",Gideon,SOTXVAO12D021935FC
31785,z,my morning jacket,7.6,"Here's an album that begs for vinyl, although ...",Off the Record,SOEXLGB12D021923A6
31786,z,my morning jacket,7.6,"Here's an album that begs for vinyl, although ...",Knot Comes Loose,SOUCXKA12AB0189194
31787,z,my morning jacket,7.6,"Here's an album that begs for vinyl, although ...",It Beats 4 U,SOGAFJB12D021923BF


In [ ]:
merged_df.to_csv('merged_df.csv')

files.download('merged_df.csv')

# 텍스트데이터 전처리

## 데이터 불러오기

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/pitchfork-2.csv'

review = pd.read_csv(filepath)

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/user_data-2.csv'

playlist = pd.read_csv(filepath)

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/info-2.csv'

info = pd.read_csv(filepath)

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/merged_df.csv'

merged_df = pd.read_csv(filepath)

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/predict_k.csv'

predict_artist = pd.read_csv(filepath)

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/selected_spotify.csv'

spotify = pd.read_csv(filepath)

In [ ]:
print(playlist.shape)
playlist.head()

(48373586, 4)


,Unnamed: 0,user_id,song_id,play_counts
0,31292919,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1
1,31292934,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1
2,31292932,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,32
3,31292931,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1
4,31292930,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,56


In [ ]:
playlist['play_counts'].describe()

count    4.837359e+07
mean     2.866859e+00
std      6.437725e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      9.667000e+03
Name: play_counts, dtype: float64

## 전처리

In [ ]:
playlist = playlist.drop(columns=['Unnamed: 0'], axis=1)

### 재생횟수 binning

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 100]
labels = [1,2,3,4,5,6]
playlist['binned'] = pd.cut(playlist['play_counts'], bins=bins, labels = labels)

In [ ]:
playlist.head()

,user_id,song_id,play_counts,binned
0,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1,1
1,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1,1
2,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,32,4
3,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1,1
4,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,56,6


In [ ]:
playlist = playlist.drop(['play_counts'], axis=1)

In [ ]:
playlist = playlist[:50000]

In [ ]:
playlist.head()

,user_id,song_id,binned
0,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1
1,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1
2,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,4
3,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1
4,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,6


In [ ]:
playlist.to_csv('playlist.csv')

files.download('playlist.csv')

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/playlist.csv'

playlist = pd.read_csv(filepath)

In [ ]:
playlist

,Unnamed: 0,user_id,song_id,binned
0,0,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1.0
1,1,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1.0
2,2,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,4.0
3,3,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1.0
4,4,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,6.0
...,...,...,...,...
49995,49995,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGNELN12A67ADA796,1.0
49996,49996,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGLBDH12AB018847B,1.0
49997,49997,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGKKXI12AF72A319A,1.0
49998,49998,ffc1b96de28a52f8161cb1a3a119222c319ece93,SODTWKY12AF729FE39,1.0


In [ ]:
playlist = playlist.drop(columns=['Unnamed: 0'], axis=1)

In [ ]:
playlist

,user_id,song_id,binned
0,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1.0
1,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1.0
2,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,4.0
3,fffff9534445f481b6ab91c345500083d2ce4df1,SONHWUN12AC468C014,1.0
4,fffff9534445f481b6ab91c345500083d2ce4df1,SOMCMKG12A8C1347BF,6.0
...,...,...,...
49995,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGNELN12A67ADA796,1.0
49996,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGLBDH12AB018847B,1.0
49997,ffc1b96de28a52f8161cb1a3a119222c319ece93,SOGKKXI12AF72A319A,1.0
49998,ffc1b96de28a52f8161cb1a3a119222c319ece93,SODTWKY12AF729FE39,1.0


In [ ]:
playlist['binned'].describe()

count    49969.000000
mean         1.073486
std          0.406484
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: binned, dtype: float64

In [ ]:
print(playlist.isnull().sum())

user_id     0
song_id     0
binned     31
dtype: int64


In [ ]:
playlist = playlist.dropna()

In [ ]:
merged_df = pd.merge(playlist, info, on=['song_id'], how='inner')

In [ ]:
merged_df.head()

,user_id,song_id,binned,Unnamed: 0,title,artist,album
0,fffff9534445f481b6ab91c345500083d2ce4df1,SOFYCAM12A58A7A45B,1,74283,I Wanna Make You Close Your Eyes,dierks bentley,feel that fire
1,ffcc413ce121c5056cabb22605b9c645164e4ad9,SOFYCAM12A58A7A45B,1,74283,I Wanna Make You Close Your Eyes,dierks bentley,feel that fire
2,ffca7eb4f65b004d6897d3da4689aa3cee72a5d4,SOFYCAM12A58A7A45B,1,74283,I Wanna Make You Close Your Eyes,dierks bentley,feel that fire
3,fffff9534445f481b6ab91c345500083d2ce4df1,SOOOWXV12A6D4F4763,1,325856,Train Travelin',dierks bentley,dierks bentley
4,fffff9534445f481b6ab91c345500083d2ce4df1,SONYEOJ12A8C142E86,4,187566,Breed,nirvana,with the lights out - box set


In [ ]:
# merged_df = merged_df.drop(columns=['Unnamed: 0', 'title', 'album'],axis = 1)
merged_df = merged_df.drop(columns=['song_id'],axis = 1)

In [ ]:
playlist.shape

(1000000, 3)

In [ ]:
review.head()

,Unnamed: 0,title,artist,score,content
0,0,pure tone audiometry,aarktica,7.4,Can there be any purpose behind a master's deg...
1,1,or you could just go through your whole life a...,aarktica,8.1,Funny what gets tagged for popular comebacks t...
2,2,bleeding light,aarktica,7.2,Aarktica didn't seem to know what he was on 20...
3,3,abcs,abcs,8.0,ABCS are all about unity. Their bandname come...
4,4,homesick and happy to be here,aberdeen,7.0,"If you caught this little blip in the mid-90s,..."


In [ ]:
review.shape

(9848, 5)

In [ ]:
len(review['artist'].unique())

5130

pitchfork 리뷰는 총 9849개이지만 리뷰를 받은 아티스트는 총 5130명이다. 그렇기 때문에 아티스트 별로 묶어 분석을 한다.

In [ ]:
info.head()

,Unnamed: 0,title,song_id,artist,album
0,0,Silent Night,SOQMMHC12AB0180CB8,faster pussy cat,monster ballads x-mas
1,1,Tanssi vaan,SOVFVAK12A8C1350D9,karkkiautomaatti,karkuteillä
2,2,No One Could Ever,SOGTUKN12AB017F4F1,hudson mohawke,butter
3,3,Si Vos Querés,SOBNYVR12A8C13558C,yerba brava,de culo
4,4,Tangle Of Aspens,SOHSBXH12A8C13B0DF,der mystic,rene ablaze presents winter sessions


In [ ]:
info.shape

(1000000, 5)

In [ ]:
whole_info = pd.merge(playlist, info, on=['song_id'], how='inner')

In [ ]:
whole_info = whole_info.sort_values(by='user_id', ascending=False)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
whole_info.head(500)

In [ ]:
whole_info.groupby(['user_id','artist'])['song_id'].count()

In [ ]:
print(review.shape)
review.head()
review.tail()
print(review.isnull().sum())

(9848, 5)
Unnamed: 0    0
title         1
artist        0
score         0
content       3
dtype: int64


In [ ]:
review = review.dropna()
print(review.isnull().sum())

Unnamed: 0    0
title         0
artist        0
score         0
content       0
dtype: int64


In [ ]:
review.head()

,Unnamed: 0,title,artist,score,content
0,0,pure tone audiometry,aarktica,7.4,Can there be any purpose behind a master's deg...
1,1,or you could just go through your whole life a...,aarktica,8.1,Funny what gets tagged for popular comebacks t...
2,2,bleeding light,aarktica,7.2,Aarktica didn't seem to know what he was on 20...
3,3,abcs,abcs,8.0,ABCS are all about unity. Their bandname come...
4,4,homesick and happy to be here,aberdeen,7.0,"If you caught this little blip in the mid-90s,..."


In [ ]:
predict_artist

,artist,predict_k
0,'n sync/phil collins,12
1,3 doors down,30
2,4 non blondes,15
3,a day to remember,30
4,adam lambert,30
...,...,...
287,weezer,12
288,white denim,11
289,yael naïm,10
290,yann tiersen,11


In [ ]:
artist_with_k = pd.merge(review, predict_artist, on=['artist'], how='inner')

In [ ]:
artist_with_k

,Unnamed: 0,title,artist,score,content,predict_k
0,158,out of exile,audioslave,6.8,"Fresh out of exile, Chris Cornell is trying to...",10
1,214,funeral,arcade fire,9.7,How did we get here?Ours is a generation overw...,14
2,215,arcade fire ep,arcade fire,6.8,"If you're an Arcade Fire fan, chances are you'...",14
3,5190,neon bible,arcade fire,8.4,Sharing its title with a John Kennedy Too...,14
4,8548,the suburbs,arcade fire,8.6,Arcade Fire never aim for anything less than g...,14
...,...,...,...,...,...,...
266,8434,something for everybody,devo,6.6,There are a few bad omens hovering around Some...,11
267,8551,outside the box,skream,7.9,"If you're Ollie Jones, a musician who's made h...",27
268,8835,man on the moon ii: the legend of mr. rager,kid cudi,6.7,Kanye West has repeatedly called kiD CuDi his ...,10
269,9394,virtue,emmy the great,6.3,Virtue's cover art includes toy dinosaurs and ...,30


### 리뷰에서 앨범명 제거 

In [ ]:
# 리뷰 내용 속 앨범명 없애기
artist_with_k['content'] = artist_with_k.content.str.lower() # 앨범명과 같이 모두 소문자로 변환

In [ ]:
artist_with_k.head()

,Unnamed: 0,title,artist,score,content,predict_k
0,158,out of exile,audioslave,6.8,"fresh out of exile, chris cornell is trying to...",10
1,214,funeral,arcade fire,9.7,how did we get here?ours is a generation overw...,14
2,215,arcade fire ep,arcade fire,6.8,"if you're an arcade fire fan, chances are you'...",14
3,5190,neon bible,arcade fire,8.4,sharing its title with a john kennedy too...,14
4,8548,the suburbs,arcade fire,8.6,arcade fire never aim for anything less than g...,14


In [ ]:
for i in range(artist_with_k.shape[0]):
  value_to_replace = artist_with_k['title'].iloc[i] # 앨범명을 변수에 저장
  # 리뷰 내용에 앨범명이 있는 경우 빈 칸으로 대체
  artist_with_k = artist_with_k.replace({'content' : value_to_replace}, ' ')



In [ ]:
artist_with_k['content'].iloc[0]

In [ ]:
artist_with_k.to_csv('review_edited.csv')

files.download('review_edited.csv')

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/review_edited.csv'

review = pd.read_csv(filepath)

In [ ]:
review_edited = artist_with_k.drop(columns=['Unnamed: 0'],axis = 1)

### accented character 제거하기 

In [ ]:
review_edited['content'] = review_edited['content'].apply(unidecode)

### 불용어 추가 및 제거 

In [ ]:
# 아티스트명을 불용어 처리하기
# 리뷰 데이터프레임의 아티스트명을 저장한다
artists = review_edited['artist'].unique()

In [ ]:
artists = list(artists)

In [ ]:
type(artists)

list

In [ ]:
artists.extend(['/n', '/n/n']) # 불용어에 줄바꿈 명령어 추가

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
print(nlp.Defaults.stop_words)

{'whence', 'others', 'whither', 'be', 'too', 'can', 'except', 'only', 'make', 'cannot', 'done', 'with', 'thereafter', 'elsewhere', 'hundred', 'whose', 'give', 'his', 'next', 'herself', 'down', "n't", 'it', 'n’t', 'whereas', 'many', 'own', 'themselves', 'yourself', 'moreover', 'something', 'beforehand', 'serious', 'other', 'both', 'fifty', "'re", 're', 'get', 'put', 'became', 'over', 'anywhere', 'just', 'amongst', 'a', 'show', 'formerly', 'sometimes', '‘ve', 'us', 'whereby', 'wherever', 'even', 'much', 'regarding', 'see', '’re', 'using', 'nine', 'few', 'beyond', 'fifteen', 'several', 'must', 'side', 'been', 'becomes', 'still', 'and', 'not', 'between', 'never', 'onto', 'those', 'why', 'among', 'former', 'then', 'however', 'yet', 'thence', 'thereupon', 'very', 'already', 'i', 'now', 'eight', 'here', 'besides', 'enough', "'s", 'back', 'anyway', 'that', 'nor', 'say', 'each', 'might', 'himself', 'through', 'because', 'hereupon', 'ours', 'anyone', 'always', 'made', 'first', 'ten', 'n‘t', 'any

In [ ]:
for name in artists:
  nlp.Defaults.stop_words.add(name)

In [ ]:
nlp.Defaults.stop_words

## 토큰화

In [ ]:
def spacy_tokenizer(document):
    tokens = nlp(document)
    tokens = [token.lemma_ for token in tokens if (
        token.is_stop == False and \
        token.is_punct == False and \
        token.lemma_.strip()!= '')]
    tokens = map(unidecode, tokens)
    return tokens


## 벡터화

In [ ]:
tfidf_vectorizer = \
    TfidfVectorizer(tokenizer = spacy_tokenizer, binary=True)

In [ ]:
result = tfidf_vectorizer.fit_transform(review_edited['content'])

In [ ]:
tfidf_vectorizer.get_feature_names()

In [ ]:
review_edited

,title,artist,score,content,predict_k
0,out of exile,audioslave,6.8,"fresh out of exile, chris cornell is trying to...",10
1,funeral,arcade fire,9.7,how did we get here?ours is a generation overw...,14
2,arcade fire ep,arcade fire,6.8,"if you're an arcade fire fan, chances are you'...",14
3,neon bible,arcade fire,8.4,sharing its title with a john kennedy too...,14
4,the suburbs,arcade fire,8.6,arcade fire never aim for anything less than g...,14
...,...,...,...,...,...
266,something for everybody,devo,6.6,there are a few bad omens hovering around some...,11
267,outside the box,skream,7.9,"if you're ollie jones, a musician who's made h...",27
268,man on the moon ii: the legend of mr. rager,kid cudi,6.7,kanye west has repeatedly called kid cudi his ...,10
269,virtue,emmy the great,6.3,virtue's cover art includes toy dinosaurs and ...,30


# 데이터 라벨링

In [ ]:
reader = Reader(rating_scale=(1, 6))
data = Dataset.load_from_df(playlist[['user_id', 'song_id', 'binned']], reader)
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
knn = KNNBasic(k=30, min_k = 10, sim_options=sim_options, verbos=True)

In [ ]:
kf = KFold(n_splits=3)

In [ ]:
for trainset, testset in kf.split(data):

    knn.fit(trainset)
    predictions = knn.test(testset)

    accuracy.rmse(predictions, verbose=True)

Computing the cosine similarity matrix...


In [ ]:
predictions

In [ ]:
column_names = ['song_id', 'predict_k']

df = pd.DataFrame(columns = column_names)

In [ ]:
for i in range(len(predictions)):
  if predictions[i][4]['was_impossible'] == False:
    id, predict = predictions[i][1], predictions[i][4]['actual_k']
    df = df.append({'song_id':id, 'predict_k':predict}, ignore_index=True)

In [ ]:
df

In [ ]:
df.to_csv('predictions.csv')

files.download('predictions.csv')

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/predictions.csv'

predictions = pd.read_csv(filepath)

In [ ]:
merged_df = pd.merge(predictions, info, on=['song_id'], how='inner')

In [ ]:
merged_df

In [ ]:
len(merged_df['artist'].unique())

292

In [ ]:
artist_list = merged_df['artist'].unique()

In [ ]:
df = merged_df.groupby('artist')['predict_k'].agg(lambda x: x.value_counts().index[0])

In [ ]:
df

artist
'n sync/phil collins    12
3 doors down            30
4 non blondes           15
a day to remember       30
adam lambert            30
                        ..
weezer                  12
white denim             11
yael naïm               10
yann tiersen            11
yeah yeah yeahs         30
Name: predict_k, Length: 292, dtype: int64

In [ ]:
df.to_csv('predict_k.csv')

files.download('predict_k.csv')

In [ ]:
filepath = '/content/drive/MyDrive/capstone project 1/predict_k.csv'

predictions = pd.read_csv(filepath)

In [ ]:
predictions['predict_k'].value_counts(normalize=True)

30    0.239726
10    0.184932
11    0.095890
12    0.089041
13    0.058219
15    0.054795
19    0.044521
14    0.041096
23    0.030822
22    0.027397
18    0.023973
20    0.020548
16    0.017123
17    0.013699
29    0.013699
21    0.013699
24    0.013699
27    0.006849
25    0.003425
26    0.003425
28    0.003425
Name: predict_k, dtype: float64

In [ ]:
columns = ['Unnamed: 0', 'duration_ms', 'count', 'popularity']

In [ ]:
spotify = spotify.drop(columns=columns, axis=1)

In [ ]:
spotify = spotify.rename(columns={'artists':'artist'})

In [ ]:
spotify

,artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode
0,aceyalone,0.142742,0.814000,0.657091,0.000056,0.108891,-7.763091,0.303536,94.308818,0.645818,6,1
1,aceyalone,0.142742,0.814000,0.657091,0.000056,0.108891,-7.763091,0.303536,94.308818,0.645818,6,1
2,aceyalone,0.142742,0.814000,0.657091,0.000056,0.108891,-7.763091,0.303536,94.308818,0.645818,6,1
3,ryan adams,0.388517,0.480500,0.299500,0.001517,0.156067,-11.452833,0.035750,124.993167,0.298667,7,1
4,ryan adams & the cardinals,0.082300,0.434000,0.843000,0.000463,0.216000,-4.916000,0.031500,154.226000,0.890000,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3052,the roots,0.091760,0.779519,0.608852,0.007072,0.246689,-7.548630,0.302137,99.233407,0.633704,1,1
3053,throbbing gristle,0.388040,0.474750,0.234750,0.774250,0.089000,-25.171500,0.127825,92.938500,0.421875,11,1
3054,various artists,0.525641,0.503986,0.509906,0.205204,0.268709,-12.913222,0.124518,117.311278,0.360840,7,1
3055,amy winehouse,0.266410,0.621808,0.624115,0.003170,0.152862,-5.529462,0.058212,111.585692,0.542731,9,1


In [ ]:
spotify_merged = pd.merge(spotify, df, on=['artist'], how='inner')

In [ ]:
spotify_merged 

,artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,predict_k
0,audioslave,0.037588,0.445471,0.730882,0.004222,0.187182,-4.856353,0.045112,104.802471,0.407353,9,1,10
1,arcade fire,0.118172,0.382778,0.785667,0.111533,0.184778,-6.139000,0.033711,123.824444,0.373444,0,1,14
2,arcade fire,0.118172,0.382778,0.785667,0.111533,0.184778,-6.139000,0.033711,123.824444,0.373444,0,1,14
3,arcade fire,0.118172,0.382778,0.785667,0.111533,0.184778,-6.139000,0.033711,123.824444,0.373444,0,1,14
4,arcade fire,0.118172,0.382778,0.785667,0.111533,0.184778,-6.139000,0.033711,123.824444,0.373444,0,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,lady gaga,0.091975,0.690207,0.754598,0.001870,0.212476,-4.981586,0.056539,120.065897,0.579057,0,1,15
223,foals,0.017200,0.567000,0.837500,0.039379,0.356000,-6.856500,0.041850,116.048500,0.721000,2,0,12
224,devo,0.191829,0.674817,0.645233,0.176667,0.129728,-11.910883,0.076715,143.060617,0.760983,9,1,11
225,kid cudi,0.341685,0.657643,0.650416,0.124871,0.254503,-7.190981,0.091326,122.937597,0.463404,0,0,10


In [ ]:
spotify_merged = spotify_merged.drop_duplicates(['artist'], keep='last')

In [ ]:
spotify_merged

,artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,predict_k
0,audioslave,0.037588,0.445471,0.730882,0.004222,0.187182,-4.856353,0.045112,104.802471,0.407353,9,1,10
4,arcade fire,0.118172,0.382778,0.785667,0.111533,0.184778,-6.139000,0.033711,123.824444,0.373444,0,1,14
12,beastie boys,0.149497,0.703967,0.729819,0.154794,0.232938,-9.559016,0.211940,122.726044,0.645319,1,1,19
18,the black keys,0.298796,0.491786,0.710357,0.144837,0.190889,-6.250179,0.058661,125.873071,0.523286,9,0,10
24,bright eyes,0.554333,0.460222,0.456956,0.045800,0.173967,-12.285556,0.052467,102.256444,0.399256,0,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,lady gaga,0.091975,0.690207,0.754598,0.001870,0.212476,-4.981586,0.056539,120.065897,0.579057,0,1,15
223,foals,0.017200,0.567000,0.837500,0.039379,0.356000,-6.856500,0.041850,116.048500,0.721000,2,0,12
224,devo,0.191829,0.674817,0.645233,0.176667,0.129728,-11.910883,0.076715,143.060617,0.760983,9,1,11
225,kid cudi,0.341685,0.657643,0.650416,0.124871,0.254503,-7.190981,0.091326,122.937597,0.463404,0,0,10


# Random Forest

## Pitchfork

In [ ]:
input_train, input_eval, label_train, label_eval = \
    train_test_split(result, review_edited['predict_k'], test_size=0.1, random_state=42)

In [ ]:
pipe = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=30,random_state=42, oob_score=True)
)

In [ ]:
pipe.fit(input_train, label_train)
print('검증 정확도', pipe.score(input_eval, label_eval))

검증 정확도 0.21428571428571427


## Spotify

In [ ]:
target = 'predict_k'

In [ ]:
features = spotify_merged.drop(columns=[target]).columns

In [ ]:
features = features[1:]

In [ ]:
features

Index(['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'key',
       'mode'],
      dtype='object')

In [ ]:
pipe = make_pipeline(
    PCA(n_components=2),
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=30,random_state=42, oob_score=True)
)

In [ ]:
input_train, input_eval, label_train, label_eval = \
    train_test_split(spotify_merged[features], spotify_merged[target], test_size=0.1, random_state=42)

In [ ]:
pipe.fit(input_train, label_train)
print('검증 정확도', pipe.score(input_eval, label_eval))

검증 정확도 0.14285714285714285
